In [63]:
import os
if not 'notebooks' in os.listdir():
    os.chdir('..')
print(os.listdir())

['.gitignore', 'classification_image.py', 'sparse', 'requirements.txt', '.git', 'hpc', 'laplace', '__pycache__', 'data_utils', 'tmp', '.ipynb_checkpoints', 'scripts', 'results', 'asdfghjkl', 'notebooks', 'configs', 'data', 'arg_utils.py']


In [129]:
import torch
from torch import nn
from pathlib import Path
import numpy as np
import pickle
import torch.nn.functional as F

In [130]:
print(os.listdir('results/mnist/mixer1/baseline_approx=kron_fixedaug_E=2_N=50000_S=31_seed=711'))
path = Path('results/mnist/mixer1/baseline_approx=kron_fixedaug_E=2_N=50000_S=31_seed=711')

FileNotFoundError: [Errno 2] No such file or directory: 'results/mnist/mixer1/baseline_approx=kron_fixedaug_E=2_N=50000_S=31_seed=711'

In [131]:
def get_weights(path):
    
    weights = {}
    for name in os.listdir(path):
        with open(path / name, 'rb') as f:
            data = pickle.load(f)
        weight = data.detach().cpu().numpy()
        
        weights[name] = weight
    return weights
        
weights = get_weights()


TypeError: get_weights() missing 1 required positional argument: 'path'

In [132]:
keys = [n for n in weights.keys()]

keys_z = [x for x in keys if 'conv_net.0' in x]
print(keys_z)

['conv_net.0.linear.lin_V.weight.pkl', 'conv_net.0.linear.lin_V.bias.pkl', 'conv_net.0.gconv.conv.bias.pkl', 'conv_net.0.linear.lin_U.weight.pkl', 'conv_net.0.linear.lin_U.bias.pkl', 'conv_net.0.conv.bias.pkl', 'conv_net.0.conv.weight.pkl', 'conv_net.0.gconv.conv.weight.pkl']


In [157]:
"""
The function `torch_conv_layer_to_affine` takes a `torch.nn.Conv2d` layer `conv`
and produces an equivalent `torch.nn.Linear` layer `fc`.
Specifically, this means that the following holds for `x` of a valid shape:
    torch.flatten(conv(x)) == fc(torch.flatten(x))
Or equivalently:
    conv(x) == fc(torch.flatten(x)).reshape(conv(x).shape)
allowing of course for some floating-point error.
"""
from typing import Tuple

import torch
import torch.nn as nn
import numpy as np


def torch_conv_layer_to_affine(
    conv: torch.nn.Conv2d, input_size: Tuple[int, int]
) -> torch.nn.Linear:
    w, h = input_size

    # Formula from the Torch docs:
    # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
    output_size = [
        (input_size[i] + 2 * conv.padding[i] - conv.kernel_size[i]) // conv.stride[i]
        + 1
        for i in [0, 1]
    ]

    in_shape = (conv.in_channels, w, h)
    out_shape = (conv.out_channels, output_size[0], output_size[1])

    fc = nn.Linear(in_features=np.product(in_shape), out_features=np.product(out_shape))
    fc.weight.data.fill_(0.0)

    # Output coordinates
    for xo, yo in range2d(output_size[0], output_size[1]):
        # The upper-left corner of the filter in the input tensor
        xi0 = -conv.padding[0] + conv.stride[0] * xo
        yi0 = -conv.padding[1] + conv.stride[1] * yo

        # Position within the filter
        for xd, yd in range2d(conv.kernel_size[0], conv.kernel_size[1]):
            # Output channel
            for co in range(conv.out_channels):
                #fc.bias[enc_tuple((co, xo, yo), out_shape)] = conv.bias[co]
                for ci in range(conv.in_channels):
                    # Make sure we are within the input image (and not in the padding)
                    if 0 <= xi0 + xd < w and 0 <= yi0 + yd < h:
                        cw = conv.weight[co, ci, xd, yd]
                        # Flatten the weight position to 1d in "canonical ordering",
                        # i.e. guaranteeing that:
                        # FC(img.reshape(-1)) == Conv(img).reshape(-1)
                        fc.weight[
                            enc_tuple((co, xo, yo), out_shape),
                            enc_tuple((ci, xi0 + xd, yi0 + yd), in_shape),
                        ] = cw

    return fc


def range2d(to_a, to_b):
    for a in range(to_a):
        for b in range(to_b):
            yield a, b


def enc_tuple(tup: Tuple, shape: Tuple) -> int:
    res = 0
    coef = 1
    for i in reversed(range(len(shape))):
        assert tup[i] < shape[i]
        res += coef * tup[i]
        coef *= shape[i]

    return res


def dec_tuple(x: int, shape: Tuple) -> Tuple:
    res = []
    for i in reversed(range(len(shape))):
        res.append(x % shape[i])
        x //= shape[i]

    return tuple(reversed(res))

In [158]:
linear = weights['conv_net.0.linear.lin_V.weight.pkl']
print(linear.shape)
conv = weights['conv_net.0.conv.weight.pkl']
print(conv.shape)


(1024, 32)
(8, 4, 3, 3)


In [163]:
linear_layer = nn.Linear(linear.shape[1], linear.shape[0], bias=False)
linear_layer.weight = torch.tensor(linear)

conv_layer = nn.Conv2d(conv.shape[1], conv.shape[0], conv.shape[-2:], bias=False)
conv_layer.weight = torch.tensor(conv)

linconv_layer = torch_conv_layer_to_affine(conv_layer, (32, 32))


TypeError: cannot assign 'torch.FloatTensor' as parameter 'weight' (torch.nn.Parameter or None expected)